In [2]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division

## download

In [9]:
# create --post-file for download
DIR = '/home/lindb/scratch/testdata/readSetLinks'
tmpfile = op.join(DIR,'.tmp.dat')
if not op.exists(tmpfile):
    with open(tmpfile,'w') as o:
        o.write("%s" % 'j_username=mlu1&j_password=rrgmmngu')
    !chmod 600 $tmpfile

In [6]:
# create dirs, open file with weblinks to fastq files
shdir = op.join(DIR,'shfiles')
if not op.exists(shdir):
    os.makedirs(shdir)
links = open(op.join(DIR,'readSetLinks.txt'),'r').readlines()
print len(links)
# make sure file name parse is correct
links[0].split("/")[-1].replace("\n","")

384


'HI.4779.005.D704---D504.LP_mg10_cap2_kit1_R2.fastq.gz'

In [16]:
# wget links
files = [f for f in fs('/home/lindb/scratch/testdata') if 'fastq' in f]
for i,link in enumerate(links):
    link = link.replace("\n","")
    fqgz = link.split("/")[-1]
    f = op.join('/home/lindb/scratch/testdata',fqgz)
    if not f in files: # faster to check if f in list than op.exists(f)
        text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=link%s
#SBATCH --export=all
#SBATCH --time=00:59:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1

cd /home/lindb/scratch/testdata
wget --no-cookies --no-check-certificate -c %s --post-file %s
''' % (str(i).zfill(3),
       link,
       tmpfile
      )
#         filE = op.join(shdir,'wget%s.sh' % str(i).zfill(3))
        filE = op.join(shdir,'wgetfinish%s.sh' % str(i).zfill(3))        
        with open(filE,'w') as o:
            o.write("%s" % text)

In [2]:
DIR = '/home/lindb/scratch/testdata/readSetLinks'
shdir = op.join(DIR,'shfiles')
fs(shdir)[0]

'/home/lindb/scratch/testdata/readSetLinks/shfiles/wget000.sh'

In [3]:
# sbatch with master
text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=wgetmaster
#SBATCH --export=all
#SBATCH --time=00:59:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1


'''
for f in fs(shdir):
    newtext = '''sbatch %s
''' % f
    text = text + newtext
filE=op.join(shdir,'master.sh')
with open(filE,'w') as o:
    o.write("%s" % text)

## check md5

In [20]:
# log to logfile
text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=md5sum
#SBATCH --export=all
#SBATCH --time=03:00:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1


cd /home/lindb/scratch/testdata/
md5sum -c *md5 > md5log.txt
'''
filE = op.join(shdir,'md5sumcheck.sh')
print filE
with open(filE,'w') as o:
    o.write("%s" % text)
!sbatch $fiLE

/home/lindb/scratch/testdata/readSetLinks/shfiles/md5sumcheck.sh
^C


In [7]:
# read the log, look for bad md5 checks
md5 = open('/home/lindb/scratch/testdata/md5log.txt').readlines()
for line in md5:
    res = line.split(" ")[1].split("\n")[0]
    if not res == "OK":
        print line
# one res in testdata was not ok, but I had gunzip-ed it

HI.4779.007.D704---D507.JP_101_2_cap3_kit1_R1.fastq.gz: FAILED open or read



## split files by library

In [8]:
from Bio import SeqIO

In [16]:
# get a list of fastq R1 files
files = sorted([f for f in fs('/scratch/lindb/testdata') if 'R1' in f and f.endswith('fastq.gz')])
len(files)

96

In [17]:
# import info sheet
summary = pd.read_csv('/scratch/lindb/testdata/1_CoAdapTree_Seq_summary.txt',sep='\t')
summary.head()

,Read_group_header_line,Pool_name,Sample_name,Ploidy,Index1,Index2,Adaptor_1,Adaptor_2
0,@RG\tID:mg8\tLB:LPcap2\tSM:mg8,LP_mg_cap2_kit1,LP_mg8_cap2_kit1,1,ATTACTCG,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
1,@RG\tID:mg22\tLB:LPcap2\tSM:mg22,LP_mg_cap2_kit1,LP_mg22_cap2_kit1,1,TCCGGAG,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
2,@RG\tID:mg64\tLB:LPcap2\tSM:mg64,LP_mg_cap2_kit1,LP_mg64_cap2_kit1,1,GAATTCGT,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
3,@RG\tID:mg9\tLB:LPcap2\tSM:mg9,LP_mg_cap2_kit1,LP_mg9_cap2_kit1,1,CGCTCATT,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT
4,@RG\tID:mg10\tLB:LPcap2\tSM:mg10,LP_mg_cap2_kit1,LP_mg10_cap2_kit1,1,GAGATTCC,GGCTCTGA,AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC,AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT


In [18]:
samps = sorted(summary['Sample_name'].tolist())
len(samps)

96

In [19]:
# fill in blank pool names, create dictionary for samps in pools, get a list of spp, get rg info
sampdict = {}
spp      = []
for row in summary.index:
    if type(summary.loc[row,'Pool_name']) == float:
        summary.loc[row,'Pool_name'] = summary.loc[row,'Sample_name']
    pool = summary.loc[row,'Pool_name']
    samp = summary.loc[row,'Sample_name']
    sp   = pool.split("_")[0]
    if not sp in spp:
        spp.append(sp)
        spp = sorted(spp)
    sampdict[samp] = pool


In [38]:
# get rg info
rg = '/scratch/lindb/mengmeng/4_addRG.sh'
rg = open(rg,'r').readlines()
rginfo = {}
for line in rg:
    if line.split(" ")[0] == 'java':
        splits = line.split(" ")
        samp = splits[12].split("/")[-1].split(".")[0]
        assert samp in sampdict
        if not samp in rginfo:
            rginfo[samp] = {}
        else:
            print 'duplicate samp %s' % samp
        rginfo[samp]['rgid'] = splits[6].split("=")[1]
        rginfo[samp]['rglb'] = splits[7].split("=")[1]
        rginfo[samp]['rgpl'] = splits[9].split("=")[1]
        rginfo[samp]['rgpu'] = splits[10].split("=")[1]
        rginfo[samp]['rgsm'] = splits[11].split("=")[1]
        #print samp,rginfo[samp]

In [40]:
len(rginfo.keys()) == len(sampdict.keys())

True

In [41]:
pkldump(rginfo,'/scratch/lindb/testdata/rginfo.pkl')

In [44]:
# list of files by pool
poolfdict = {}
ftopool = {}
for f in files:
    samp = f.split("---")[1].split(".")[1].replace("_R1","")
    pool = sampdict[samp]
    if not pool in poolfdict:
        poolfdict[pool] = []
    f2 = op.abspath(f).replace("_R1","_R2")
    assert op.exists(f2)
    [poolfdict[pool].append(x) for x in [f,f2]]
    ftopool[f]  = pool
    ftopool[f2] = pool
    
# poolfdict

In [17]:
samp = sampdict.keys()[0]
sampdict[samp]

'JP_cap4_kit1'

## parse fastq files for efficiency

In [3]:
# multiprocess count files 
rc = Client(profile="job_10804682")
dview = rc[:]
lview = rc.load_balanced_view()
len(lview)

200

In [57]:
def countreads(f):
    linecount = !zcat $f | wc -l | bc
    readcount = int(linecount[0]) / 4
    return (f,readcount)
dview['countreads'] = countreads

In [103]:
allfiles = []
for f in files:
    f2 = f.replace("_R1","_R2")
    assert op.exists(f2)
    [allfiles.append(x) for x in [f,f2]]
counts = dview.map(countreads,allfiles)
len(counts)

192

In [129]:
poolreads = {}
freads = {}
for c in counts:
    f = c[0]
    freads[f] = c[1]
    pool = ftopool[f]
    if not pool in poolreads:
        poolreads[pool] = 0
    poolreads[pool] += c[1]
poolreads

{'DF_cap3_kit1': 133352664.0,
 'DF_cap4_kit1': 127930938.0,
 'DF_cap5_kit1': 158503122.0,
 'DF_p27_cap2_kit2': 154428352.0,
 'DF_p52_cap1_kit2': 109897366.0,
 'JP_cap3_kit1': 125895818.0,
 'JP_cap4_kit1': 127292702.0,
 'JP_cap5_kit1': 139053014.0,
 'JP_p101_cap1_kit1': 152988120.0,
 'JP_p102_cap2_kit1': 128065604.0,
 'LP_mg_cap2_kit1': 133595514.0,
 'LP_mg_cap3_kit1': 122122024.0,
 'LP_mg_cap4_kit1': 97997102.0,
 'LP_mg_cap5_kit1': 89972752.0,
 'LP_mg_cap6_kit1': 104349450.0,
 'LP_pmg40_cap1_kit1': 126757062.0,
 'WL_cap3_kit1': 109160064.0,
 'WL_cap4_kit1': 107461138.0,
 'WL_cap5_kit1': 160162326.0,
 'WL_p404_cap1_kit1': 141502964.0,
 'WL_p406_cap2_kit1': 158937502.0}

In [98]:
readsums = 0
for pool in sorted(poolreads,key=poolreads.get,reverse=True):
    print pool,poolreads[pool]
    readsums += poolreads[pool]
readsums

WL_cap5_kit1 80081163.0
WL_p406_cap2_kit1 79468751.0
DF_cap5_kit1 79251561.0
DF_p27_cap2_kit2 77214176.0
JP_p101_cap1_kit1 76494060.0
WL_p404_cap1_kit1 70751482.0
JP_cap5_kit1 69526507.0
LP_mg_cap2_kit1 66797757.0
DF_cap3_kit1 66676332.0
JP_p102_cap2_kit1 64032802.0
DF_cap4_kit1 63965469.0
JP_cap4_kit1 63646351.0
LP_pmg40_cap1_kit1 63378531.0
JP_cap3_kit1 62947909.0
LP_mg_cap3_kit1 61061012.0
DF_p52_cap1_kit2 54948683.0
WL_cap3_kit1 54580032.0
WL_cap4_kit1 53730569.0
LP_mg_cap6_kit1 52174725.0
LP_mg_cap4_kit1 48998551.0
LP_mg_cap5_kit1 44986376.0


1354712799.0

In [252]:
DIR = '/scratch/lindb/testdata/'
shdir = op.join(DIR,'shfiles/parse')
if not op.exists(shdir):
    os.makedirs(shdir)
count = 0
for f in ftopool.keys():
    pool = ftopool[f]
    pooldir = op.join(DIR,pool)
    if not op.exists(pooldir):
        os.makedirs(pooldir)
    text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=parse%s
#SBATCH --export=all
#SBATCH --time=08:00:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1

source $HOME/.bashrc
cd $HOME/pipeline

python 00_parse_fastq.py %s %s
''' % (str(count).zfill(3),
       f,
       pooldir
      )
    filE = op.join(shdir,'parse_%s.sh' % str(count).zfill(3))
    with open(filE,'w') as o:
        o.write("%s" % text)
    count += 1

In [171]:
os.chdir(shdir)
for f in fs(shdir):
    !sbatch $f

Submitted batch job 10807029
Submitted batch job 10807030
Submitted batch job 10807032
Submitted batch job 10807033
Submitted batch job 10807034
Submitted batch job 10807035
Submitted batch job 10807036
Submitted batch job 10807037
Submitted batch job 10807038
Submitted batch job 10807039
Submitted batch job 10807040
Submitted batch job 10807041
Submitted batch job 10807042
Submitted batch job 10807043
Submitted batch job 10807044
Submitted batch job 10807045
Submitted batch job 10807046
Submitted batch job 10807047
Submitted batch job 10807048
Submitted batch job 10807049
Submitted batch job 10807050
Submitted batch job 10807051
Submitted batch job 10807052
Submitted batch job 10807053
Submitted batch job 10807054
Submitted batch job 10807055
Submitted batch job 10807056
Submitted batch job 10807057
Submitted batch job 10807058
Submitted batch job 10807059
Submitted batch job 10807060
Submitted batch job 10807061
Submitted batch job 10807062
Submitted batch job 10807063
Submitted batc

In [175]:
for p in poolreads:
    print pool
    pdir = op.join(DIR,p)
    print len(ls(pdir))

DF_cap3_kit1
1089
DF_cap3_kit1
6643
DF_cap3_kit1
6230
DF_cap3_kit1
6002
DF_cap3_kit1
1678
DF_cap3_kit1
1029
DF_cap3_kit1
6390
DF_cap3_kit1
7440
DF_cap3_kit1
5014
DF_cap3_kit1
4038
DF_cap3_kit1
4067
DF_cap3_kit1
5476
DF_cap3_kit1
4164
DF_cap3_kit1
1159
DF_cap3_kit1
805
DF_cap3_kit1
1784
DF_cap3_kit1
6461
DF_cap3_kit1
902
DF_cap3_kit1
6339
DF_cap3_kit1
7519
DF_cap3_kit1
5801


## count files to decide lensh for 01a_trim-fastq

In [ ]:
1+1

In [3]:
# get nfiles per dir
os.chdir('/scratch/lindb/testdata/')
dirs = !ls -d */
dirs = [op.abspath(d) for d in dirs if 'kit' in d]

fcount = 0
dcount = {}
for d in dirs:
    fcount += len(ls(d))
    dcount[d] = len(ls(d))
fcount

271036

In [4]:
# determine frac of sh files
sums = sum(dcount.values())
frac = {}
for d in dcount:
    frac[d] = dcount[d]/sums
sum(frac.values()),frac

(0.9999999999999999,
 {'/scratch/lindb/testdata/DF_cap3_kit1': 0.04921855399282752,
  '/scratch/lindb/testdata/DF_cap4_kit1': 0.0472188196401954,
  '/scratch/lindb/testdata/DF_cap5_kit1': 0.058501453681429776,
  '/scratch/lindb/testdata/DF_p27_cap2_kit2': 0.05698136040968727,
  '/scratch/lindb/testdata/DF_p52_cap1_kit2': 0.040548119069053555,
  '/scratch/lindb/testdata/JP_cap3_kit1': 0.046466152097876294,
  '/scratch/lindb/testdata/JP_cap4_kit1': 0.046997446833630954,
  '/scratch/lindb/testdata/JP_cap5_kit1': 0.05132897474874187,
  '/scratch/lindb/testdata/JP_p101_cap1_kit1': 0.05645006567393261,
  '/scratch/lindb/testdata/JP_p102_cap2_kit1': 0.04725571510795614,
  '/scratch/lindb/testdata/LP_mg_cap2_kit1': 0.0493071031154533,
  '/scratch/lindb/testdata/LP_mg_cap3_kit1': 0.045078882510072464,
  '/scratch/lindb/testdata/LP_mg_cap4_kit1': 0.03617969568618191,
  '/scratch/lindb/testdata/LP_mg_cap5_kit1': 0.03322067917177054,
  '/scratch/lindb/testdata/LP_mg_cap6_kit1': 0.03852624743576499

In [5]:
# num sh files per dir
shfrac = {}
for d in frac:
    shfrac[d] = int(math.ceil(frac[d]*975))
    print d, shfrac[d]

/scratch/lindb/testdata/LP_mg_cap5_kit1 33
/scratch/lindb/testdata/WL_cap5_kit1 58
/scratch/lindb/testdata/WL_p406_cap2_kit1 58
/scratch/lindb/testdata/WL_cap3_kit1 40
/scratch/lindb/testdata/WL_cap4_kit1 39
/scratch/lindb/testdata/LP_mg_cap4_kit1 36
/scratch/lindb/testdata/JP_cap3_kit1 46
/scratch/lindb/testdata/LP_mg_cap6_kit1 38
/scratch/lindb/testdata/LP_mg_cap3_kit1 44
/scratch/lindb/testdata/LP_mg_cap2_kit1 49
/scratch/lindb/testdata/LP_pmg40_cap1_kit1 46
/scratch/lindb/testdata/WL_p404_cap1_kit1 51
/scratch/lindb/testdata/DF_cap4_kit1 47
/scratch/lindb/testdata/JP_p102_cap2_kit1 47
/scratch/lindb/testdata/DF_cap5_kit1 58
/scratch/lindb/testdata/JP_p101_cap1_kit1 56
/scratch/lindb/testdata/JP_cap4_kit1 46
/scratch/lindb/testdata/DF_p27_cap2_kit2 56
/scratch/lindb/testdata/JP_cap5_kit1 51
/scratch/lindb/testdata/DF_p52_cap1_kit2 40
/scratch/lindb/testdata/DF_cap3_kit1 48


In [46]:
shdir = '/scratch/lindb/testdata/shfiles/start_trim'
if not op.exists(shdir):
    os.makedirs(shdir)
for d in shfrac:
    text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=%s
#SBATCH --export=all
#SBATCH --time=12:00:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1
#SBATCH --mail-user=brandon.lind@ubc.ca
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL


source $HOME/.bashrc
cd $HOME/pipeline

python 01a_trim-fastq.py %s %s %s
''' % (op.basename(d),
       d,
       '/home/lindb/scratch/ptaeda.v1.01.reduced.pseudo.fasta',
       str(shfrac[d])
      )
    filE = op.join(shdir,'start-trim_%s.sh' % op.basename(d))
    with open(filE,'w') as o:
        o.write("%s" % text)

In [48]:
for f in fs(shdir):
    !sbatch $f

Submitted batch job 10822585
Submitted batch job 10822587
Submitted batch job 10822590
Submitted batch job 10822592
Submitted batch job 10822594
Submitted batch job 10822597
Submitted batch job 10822599
Submitted batch job 10822603
Submitted batch job 10822606
Submitted batch job 10822608
Submitted batch job 10822612
Submitted batch job 10822614
Submitted batch job 10822617
Submitted batch job 10822619
Submitted batch job 10822621
Submitted batch job 10822623
Submitted batch job 10822628
Submitted batch job 10822631
Submitted batch job 10822633
Submitted batch job 10822635
Submitted batch job 10822638


In [50]:
os.getcwd()

'/scratch/lindb/testdata'

In [37]:
text = !squeue -u lindb

In [38]:
count = 0
pids = []
for line in text[1:]:
#     print line.split(" ")[0]
    pids.append(line.split(" ")[0])
len(pids)

0

In [39]:
filE = '/home/lindb/scancel.txt'
with open(filE,'w') as o:
    for p in pids:
        p = int(p)
        o.write("scancel %i\n" % p)

In [36]:
!cat /home/lindb/scancel.txt | parallel --eta

Academic tradition requires you to cite works you base your article on.
If you use programs that use GNU Parallel to process data for an article in a
scientific publication, please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

To silence this citation notice: run 'parallel --citation' once.


Computers / CPU cores / Max jobs to run
1:local / 32 / 32

Computer:jobs running/jobs completed/%of started jobs/Average seconds to complete
ETA: 0s Left: 0 AVG: 0.10s  local:0/3372/100%/0.1s s     


In [8]:
## change parsed file names
os.chdir('/home/lindb/scratch/testdata/')
dirs = !ls -d */
dirs = [op.abspath(d) for d in dirs if 'kit' in d]
len(dirs),dirs

(21,
 ['/scratch/lindb/testdata/DF_cap3_kit1',
  '/scratch/lindb/testdata/DF_cap4_kit1',
  '/scratch/lindb/testdata/DF_cap5_kit1',
  '/scratch/lindb/testdata/DF_p27_cap2_kit2',
  '/scratch/lindb/testdata/DF_p52_cap1_kit2',
  '/scratch/lindb/testdata/JP_cap3_kit1',
  '/scratch/lindb/testdata/JP_cap4_kit1',
  '/scratch/lindb/testdata/JP_cap5_kit1',
  '/scratch/lindb/testdata/JP_p101_cap1_kit1',
  '/scratch/lindb/testdata/JP_p102_cap2_kit1',
  '/scratch/lindb/testdata/LP_mg_cap2_kit1',
  '/scratch/lindb/testdata/LP_mg_cap3_kit1',
  '/scratch/lindb/testdata/LP_mg_cap4_kit1',
  '/scratch/lindb/testdata/LP_mg_cap5_kit1',
  '/scratch/lindb/testdata/LP_mg_cap6_kit1',
  '/scratch/lindb/testdata/LP_pmg40_cap1_kit1',
  '/scratch/lindb/testdata/WL_cap3_kit1',
  '/scratch/lindb/testdata/WL_cap4_kit1',
  '/scratch/lindb/testdata/WL_cap5_kit1',
  '/scratch/lindb/testdata/WL_p404_cap1_kit1',
  '/scratch/lindb/testdata/WL_p406_cap2_kit1'])

In [11]:
files = []
for d in dirs:
    [files.append(f) for f in fs(d) if 'fastq' in f]
len(files)

271036

In [12]:
files[0]

'/scratch/lindb/testdata/DF_cap3_kit1/HI.4779.008.D705---D505.DF_52_20_cap3_kit1_R1._0000.fastq'

In [16]:
def switchname(f):
    dname = op.dirname(f)
    bname = op.basename(f)
    zero,one,two,three,four,parse,suf = bname.split(".")
    parse = "parse%sparse" % parse.replace("_","")
    new   = zero+'.'+one+'.'+two+'.'+three+'.'+four+parse+'.'+suf
    newf  = op.join(dname,new)
    return newf

In [67]:
files[0]

'DF_cap3_kit1/HI.4779.008.D705---D505.DF_52_20_cap3_kit1_R1._0000.fastq'

In [71]:
print 'file  =',files[0]
switched = switchname(files[0])
print 'switc =', switched
r1out    = switched.replace(".fastq","_trimmed.fastq")
print 'r1out =', r1out
html     = r1out.replace("R1","").replace(".fastq","_R1_R2_stats")
print 'html  =', html
json     = r1out.replace("R1","").replace(".fastq","_R1_R2")
print 'json  =',json

file  = DF_cap3_kit1/HI.4779.008.D705---D505.DF_52_20_cap3_kit1_R1._0000.fastq
switc = DF_cap3_kit1/HI.4779.008.D705---D505.DF_52_20_cap3_kit1_R1parse0000parse.fastq
r1out = DF_cap3_kit1/HI.4779.008.D705---D505.DF_52_20_cap3_kit1_R1parse0000parse_trimmed.fastq
html  = DF_cap3_kit1/HI.4779.008.D705---D505.DF_52_20_cap3_kit1_parse0000parse_trimmed_R1_R2_stats
json  = DF_cap3_kit1/HI.4779.008.D705---D505.DF_52_20_cap3_kit1_parse0000parse_trimmed_R1_R2


In [17]:
fcount  = 0
tcount  = 0
shcount = 0
ceil = math.ceil(len(files)/300)
text = ''''''
for f in files:
    new = switchname(f)
    newtext = '''mv %s %s
''' % (f,new)
    text = text + newtext
    fcount += 1
    tcount += 1
    if fcount == ceil or tcount == len(files):
        shz = str(shcount).zfill(3)
        header = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=mvparse%s
#SBATCH --export=all
#SBATCH --time=02:59:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1
''' % (shz)
        text = header + text
        filE = '/scratch/lindb/changeparse/chparse_%s.sh' % shz
        with open(filE,'w') as o:
            o.write("%s" % text)
        shcount += 1
        fcount = 0
        text = ''''''

In [ ]:
sbatch('/scratch/lindb/changeparse/')

Submitted batch job 10833283
Submitted batch job 10833285
Submitted batch job 10833287
Submitted batch job 10833288
Submitted batch job 10833289
Submitted batch job 10833290
Submitted batch job 10833291
Submitted batch job 10833292
Submitted batch job 10833293
Submitted batch job 10833294
Submitted batch job 10833295
Submitted batch job 10833297
Submitted batch job 10833299
Submitted batch job 10833300
Submitted batch job 10833301
Submitted batch job 10833302
Submitted batch job 10833304
Submitted batch job 10833305
Submitted batch job 10833306
Submitted batch job 10833307
Submitted batch job 10833308
Submitted batch job 10833309
Submitted batch job 10833310
Submitted batch job 10833311
Submitted batch job 10833312
^C
Submitted batch job 10833314
^C
^C
^C
^C
^C
^C
^C
^C
^C
Submitted batch job 10833324
Submitted batch job 10833325
^C
^C
^C
^C


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/lindb/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/lindb/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/lindb/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 376, in _fixed_getinnerframes
    lines = ulinecache.getlines(file)[start:end]
  File "/home/lindb/anaconda2/lib/python2.7/site-packages/IPython/utils/ulinecache.py", line 37, in getlines
    return [l.decode(encoding, 'replace') for l in lines]
  File "/home/lindb/anaconda2/lib/python2.7/encodings/utf_8.py", line 15, in decode
    def decode(input, errors='strict'):
KeyboardInterrupt


IndexError: string index out of range

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/lindb/anaconda2/lib/python2.7/site-packages/zmq/eventloop/zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "/home/lindb/anaconda2/lib/python2.7/site-packages/tornado/stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/lindb/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/home/lindb/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/lindb/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/home/lindb/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/home/lindb/anaconda2/lib/python

In [20]:
DIR = '/scratch/lindb/changeparse/'
os.chdir(DIR)
for f in fs(DIR):
    if not '000' in f:
        if not 'slurm' in f:
            !sbatch $f

Submitted batch job 10833447
Submitted batch job 10833450
Submitted batch job 10833453
Submitted batch job 10833455
Submitted batch job 10833456
Submitted batch job 10833457
Submitted batch job 10833464
Submitted batch job 10833478
Submitted batch job 10833487
Submitted batch job 10833501
Submitted batch job 10833506
Submitted batch job 10833510
Submitted batch job 10833512
Submitted batch job 10833518
Submitted batch job 10833531
Submitted batch job 10833546
Submitted batch job 10833559
Submitted batch job 10833565
Submitted batch job 10833583
Submitted batch job 10833595
Submitted batch job 10833602
Submitted batch job 10833617
Submitted batch job 10833625
Submitted batch job 10833630
Submitted batch job 10833637
Submitted batch job 10833644
Submitted batch job 10833657
Submitted batch job 10833661
Submitted batch job 10833665
Submitted batch job 10833668
Submitted batch job 10833670
Submitted batch job 10833672
Submitted batch job 10833674
Submitted batch job 10833676
Submitted batc

Submitted batch job 10834024
Submitted batch job 10834025
Submitted batch job 10834026
Submitted batch job 10834027
Submitted batch job 10834028
Submitted batch job 10834029
Submitted batch job 10834030
Submitted batch job 10834033
Submitted batch job 10834034
Submitted batch job 10834036
Submitted batch job 10834037
Submitted batch job 10834038


In [22]:
## kick off lodgepole and jack pine
jplp = [d for d in dirs if 'LP' in d or 'JP' in d]
jplp

['/scratch/lindb/testdata/JP_cap3_kit1',
 '/scratch/lindb/testdata/JP_cap4_kit1',
 '/scratch/lindb/testdata/JP_cap5_kit1',
 '/scratch/lindb/testdata/JP_p101_cap1_kit1',
 '/scratch/lindb/testdata/JP_p102_cap2_kit1',
 '/scratch/lindb/testdata/LP_mg_cap2_kit1',
 '/scratch/lindb/testdata/LP_mg_cap3_kit1',
 '/scratch/lindb/testdata/LP_mg_cap4_kit1',
 '/scratch/lindb/testdata/LP_mg_cap5_kit1',
 '/scratch/lindb/testdata/LP_mg_cap6_kit1',
 '/scratch/lindb/testdata/LP_pmg40_cap1_kit1']

In [23]:
# get nfiles per dir
fcount = 0
dcount = {}
for d in jplp:
    fcount += len(ls(d))
    dcount[d] = len(ls(d))
fcount

134865

In [24]:
# determine frac of sh files
sums = sum(dcount.values())
frac = {}
for d in dcount:
    frac[d] = dcount[d]/sums
sum(frac.values()),frac

(1.0,
 {'/scratch/lindb/testdata/JP_cap3_kit1': 0.09340451562673785,
  '/scratch/lindb/testdata/JP_cap4_kit1': 0.09445000556111667,
  '/scratch/lindb/testdata/JP_cap5_kit1': 0.10315500685871057,
  '/scratch/lindb/testdata/JP_p101_cap1_kit1': 0.11344678011344678,
  '/scratch/lindb/testdata/JP_p102_cap2_kit1': 0.09496904311719126,
  '/scratch/lindb/testdata/LP_mg_cap2_kit1': 0.09909168427686946,
  '/scratch/lindb/testdata/LP_mg_cap3_kit1': 0.0905942980017054,
  '/scratch/lindb/testdata/LP_mg_cap4_kit1': 0.07270974678382086,
  '/scratch/lindb/testdata/LP_mg_cap5_kit1': 0.06676305935565195,
  '/scratch/lindb/testdata/LP_mg_cap6_kit1': 0.07742557372187002,
  '/scratch/lindb/testdata/LP_pmg40_cap1_kit1': 0.09399028658287918})

In [25]:
# num sh files per dir
shfrac = {}
for d in frac:
    shfrac[d] = int(math.ceil(frac[d]*975))
    print d, shfrac[d]

/scratch/lindb/testdata/LP_mg_cap5_kit1 66
/scratch/lindb/testdata/LP_mg_cap4_kit1 71
/scratch/lindb/testdata/LP_mg_cap3_kit1 89
/scratch/lindb/testdata/LP_mg_cap6_kit1 76
/scratch/lindb/testdata/JP_cap3_kit1 92
/scratch/lindb/testdata/LP_mg_cap2_kit1 97
/scratch/lindb/testdata/LP_pmg40_cap1_kit1 92
/scratch/lindb/testdata/JP_p101_cap1_kit1 111
/scratch/lindb/testdata/JP_p102_cap2_kit1 93
/scratch/lindb/testdata/JP_cap4_kit1 93
/scratch/lindb/testdata/JP_cap5_kit1 101


In [27]:
shdir = '/scratch/lindb/testdata/shfiles/start_trim'
if not op.exists(shdir):
    os.makedirs(shdir)
for d in shfrac:
    text = '''#!/bin/bash
#SBATCH --account=def-saitken
#SBATCH --job-name=%s
#SBATCH --export=all
#SBATCH --time=12:00:00
#SBATCH --mem=1000mb
#SBATCH --cpus-per-task=1
#SBATCH --mail-user=brandon.lind@ubc.ca
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL


source $HOME/.bashrc
cd $HOME/pipeline

python 01a_trim-fastq.py %s %s %s
''' % (op.basename(d),
       d,
       '/scratch/lindb/refgenomes/LP_db/ptaeda.v1.01.reduced.pseudo.fasta',
       str(shfrac[d])
      )
    filE = op.join(shdir,'start-trim_%s.sh' % op.basename(d))
    with open(filE,'w') as o:
        o.write("%s" % text)

In [30]:
for f in fs(shdir):
    !sbatch $f

Submitted batch job 10834343
Submitted batch job 10834344
Submitted batch job 10834345
Submitted batch job 10834346
Submitted batch job 10834348
Submitted batch job 10834349
Submitted batch job 10834350
Submitted batch job 10834351
Submitted batch job 10834352
Submitted batch job 10834353


# why one command?

In [3]:
fqdir = '/scratch/lindb/testdata/JP_cap3_kit1/'

In [4]:
lensh = 92

In [5]:
gzfiles = [f for f in fs(fqdir) if 'R1' in f]
lgz     = len(gzfiles)
lgz

6297

In [32]:
seq_pairs = []
for f in gzfiles:
    assert 'R1' in f
    read2 = f.replace("_R1","_R2")
    assert op.exists(read2)
    if op.exists(read2):
        seq_pairs.append((f,read2))
#     else:
#         !echo 'no pair for '$f >> $fqdir'/messages/msgs.txt' # #iheartjupyter
    else:
        text = 'no pair for %s' % f
        with open(mfile,'a') as o:
            o.write("%s\n" % text)
print "found %s R1/R2 seq pairs" % str(len(seq_pairs))

found 6297 R1/R2 seq pairs


In [7]:
if len(seq_pairs) <= lensh:
    # one command per sh file
    ceil = 1
else:
    # multiple commands per sh file
    ceil = math.ceil(len(seq_pairs)/lensh)
ceil

69.0

In [17]:
lensh

92

In [12]:
shcount = 0
fcount  = 0
tcount  = 0
text = ''''''
for s in seq_pairs:
#     print s
    r1    = op.abspath(s[0])
    if r1.endswith("fastq"):
        r1out = 'hey'
    else:
        r1out = 'hey'
    r2    = op.abspath(s[1])
    if r2.endswith("fastq"):
        r2out = 'hey'
    else:
        r2out = 'hey'
    html = 'hey'
    json = 'hey'
    cmd  = '''fastp -i %s -o %s -I %s -O %s -g --cut_window_size 5 --cut_mean_quality 30 --qualified_quality_phred 30 --unqualified_percent_limit 20 --n_base_limit 5 --length_required 75 -h %s.html --cut_by_quality3 --thread 16 --json %s.json --adapter_sequence AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC --adapter_sequence_r2 AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT


cd $HOME/pipeline
# once finished, map using bwa mem 
python 01b_bwa-map.py %s %s %s %s %s           
''' % (r1  , r1out,
       r2  , r2out,
       html, json ,
       'hi' , r1out,  r2out,  'shdir',  str(tcount).zfill(3)
      )
    text = text + cmd
    
    fcount += 1
    tcount += 1
    if fcount == ceil or tcount == len(seq_pairs):
        shz = str(shcount).zfill(3)
        shcount += 1
        fcount = 0
        text = ''''''

In [13]:
shcount

92

In [20]:
len([x for x in ls('/scratch/lindb/testdata/JP_cap4_kit1/shfiles/trimmed_shfiles/') if 'sh' in x])

6369

In [45]:
DIR = '/scratch/lindb/testdata/'
os.chdir(DIR)
dirs = !ls -d *kit*/
dirs = [op.abspath(d) for d in dirs if 'JP' in d or 'LP' in d]
dirs

['/scratch/lindb/testdata/JP_cap3_kit1',
 '/scratch/lindb/testdata/JP_cap4_kit1',
 '/scratch/lindb/testdata/JP_cap5_kit1',
 '/scratch/lindb/testdata/JP_p101_cap1_kit1',
 '/scratch/lindb/testdata/JP_p102_cap2_kit1',
 '/scratch/lindb/testdata/LP_mg_cap2_kit1',
 '/scratch/lindb/testdata/LP_mg_cap3_kit1',
 '/scratch/lindb/testdata/LP_mg_cap4_kit1',
 '/scratch/lindb/testdata/LP_mg_cap5_kit1',
 '/scratch/lindb/testdata/LP_mg_cap6_kit1',
 '/scratch/lindb/testdata/LP_pmg40_cap1_kit1']

In [49]:
for d in dirs:
    print d
    os.chdir(d)
    !rm -r -- ./*/

/scratch/lindb/testdata/JP_cap3_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/JP_cap4_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/JP_cap5_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/JP_p101_cap1_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/JP_p102_cap2_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_mg_cap2_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_mg_cap3_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_mg_cap4_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_mg_cap5_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_mg_cap6_kit1
rm: cannot remove './*/': No such file or directory
/scratch/lindb/testdata/LP_pmg40_cap1_kit1
rm: cannot remove './*/': No such file or 